In [ ]:
# Author
# Prabhu Chandhar (email: prabhu@chandhar-labs.com)


import numpy as np
from numpy import *
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt1
import matplotlib.pyplot as plt2
import matplotlib.pyplot as plt0
import matplotlib.pyplot as pltx1
import matplotlib.pyplot as pltx2
import matplotlib.pyplot as pltx3
import matplotlib.pyplot as pltx4
import math  
from scipy import signal
from scipy.fftpack import fftshift,fft,fftfreq
from scipy.io import wavfile
import librosa
import librosa.display
import soundfile as sf
from contextlib import closing
from pydub import AudioSegment
import IPython

fm_freq = 98.3e6
sample_rate = 2400000

def load_iq(filename):
    x = np.fromfile(filename, np.uint8) + np.int8(-127) #adding a signed int8 to an unsigned one results in an int16 array
    x1 =  x[::2]/128 #even numbered samples
    x2  = x[1::2]/128 #odd numbered samples
    return x1+x2*1j   #return complex samples

def graph_spectrogram(wav_file):
    data, rate = sf.read(wav_file)
    nfft = 512 # Length of each window segment
    fs = 16000 # Sampling frequencies
    noverlap = 50 # Overlap between windows
    nchannels = data.ndim
    if nchannels == 1:
        pxx, freqs, bins, im = plt.specgram(data, nfft, fs, noverlap = noverlap)
    elif nchannels == 2:
        pxx, freqs, bins, im = plt.specgram(data[:,0], nfft, fs, noverlap = noverlap)
        plt.xlabel('Time (s)')
        plt.ylabel('Frequency (Hz)')
    return pxx

In [ ]:
iq_samples = load_iq('../input/datfiles/fm_93500000.dat')

print("Processing the samples ...")

sample_rate_fm = 240000                       #decimate by 10


numtaps = 140
filter_coeff = signal.firwin(numtaps, .1*sample_rate, nyq=sample_rate, window='nuttall')
iq_comercial = convolve(iq_samples, filter_coeff, mode='same')
iq_comercial = signal.decimate(iq_comercial, sample_rate//sample_rate_fm)

angle_comercial = np.unwrap(np.angle(iq_comercial))
demodulated_comercial = np.diff(angle_comercial, axis=0)

audio_rate = 48000
audio_comercial = signal.decimate(demodulated_comercial, sample_rate_fm//audio_rate, zero_phase=True,axis=0)

audio_comercial = np.int16(1e4*audio_comercial)
wavfile.write("FM_Demodulated_output.wav", rate=audio_rate, data=audio_comercial)


In [ ]:
audio_path = './FM_Demodulated_output.wav'
IPython.display.Audio(audio_path)

In [ ]:
sam = 2000
x = range(1,sam)
x = [i/sample_rate/1e-3 for i in x]
plt.plot(x,audio_comercial[1:sam]/1e4)
plt.xlabel('Time [s]')
plt.ylabel('Amplitude')
plt.grid()
plt.show()

In [ ]:
samplerate, data = wavfile.read('./FM_Demodulated_output.wav')

datafft = fft(data)
fftabs = abs(datafft)
freqs = fftfreq(data.shape[0],1/float(samplerate))

pltx2.xlim( [10, samplerate/2] )
pltx2.xscale( 'log' )
pltx2.grid( True )
pltx2.xlabel('Frequency (Hz)')
pltx2.ylabel('PSD (Hz)')
pltx2.plot(freqs[:int(freqs.size/2)],fftabs[:int(freqs.size/2)])
pltx2.show()


In [ ]:
##### Spectrum Analysis ######

f_bd, Pxx_bd = signal.welch(iq_samples, sample_rate, detrend=lambda x: x)
f_bd, Pxx_bd = fftshift(f_bd), fftshift(Pxx_bd)

plt.semilogy((fm_freq/1e6)+(f_bd/1e6), Pxx_bd)
plt.xlabel('Frequency [MHz]')
plt.ylabel('PSD [Power/Hz]')
plt.grid()
plt.title('Before filtering')
plt.xticks(np.linspace((fm_freq/1e6)-sample_rate/2e6, (fm_freq/1e6)+sample_rate/2e6, 7))
plt.xlim((fm_freq/1e6)-sample_rate/2e6, (fm_freq/1e6)+sample_rate/2e6)
plt.show()
#plt.close()

In [ ]:
sample_rate_fm = 240000                       #decimate by 10
f_ad, Pxx_ad = signal.welch(iq_comercial, sample_rate_fm, detrend=lambda x: x)
f_ad, Pxx_ad = fftshift(f_ad), fftshift(Pxx_ad)

plt1.semilogy((fm_freq/1e6)+(f_ad/1e6), Pxx_ad)
plt1.xlabel('Frequency [MHz]')
plt1.ylabel('PSD [Power/Hz]')
plt1.grid()
plt.title('After filtering')
plt1.xticks(np.linspace((fm_freq/1e6)-sample_rate_fm/2e6, (fm_freq/1e6)+sample_rate_fm/2e6, 7))
plt1.xlim((fm_freq/1e6)-sample_rate_fm/2e6, (fm_freq/1e6)+sample_rate_fm/2e6)
plt1.show()


In [ ]:

plt2.semilogy((fm_freq/1e6)+(f_bd/1e6), Pxx_bd,label='Before filtering')
plt2.semilogy((fm_freq/1e6)+(f_ad/1e6), Pxx_ad,label='After filtering')
plt2.legend()
plt2.xlabel('Frequency [MHz]')
plt2.ylabel('PSD [Power/Hz]')
plt2.show()


In [ ]:
x = graph_spectrogram(audio_path)